In [1]:
import findspark
findspark.init()

# New Section

In [2]:
from pyspark import SparkContext
sc =SparkContext(master="local", appName="New Spark Context")

In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [5]:
spark = SparkSession(sc)

In [6]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True,header=True)

In [7]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
print((data.count(), len(data.columns)))

(500, 8)


In [9]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [10]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length','Time on App','Time on Website', 'Length of Membership'],
    outputCol='features') #inputs

In [13]:
data_pre = assembler.transform(data)

In [15]:
data_pre.select('features').show(2,False)

+--------------------------------------------------------------------------+
|features                                                                  |
+--------------------------------------------------------------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]|
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]|
+--------------------------------------------------------------------------+
only showing top 2 rows



In [16]:
final_data = data_pre.select('features','Yearly Amount Spent')

In [17]:
final_data.show(2)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
+--------------------+-------------------+
only showing top 2 rows



In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  498.7581886108429|
| stddev|  75.19727389534498|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                140|
|   mean| 500.74336592422844|
| stddev|  89.31059035800199|
|    min|   266.086340948469|
|    max|  712.3963268096637|
+-------+-------------------+



In [21]:
# Create a linear regression model object
lr = LinearRegression(featuresCol='features',
                     labelCol='Yearly Amount Spent',
                     predictionCol='Predict_Yearly Amount Spent')

In [22]:
#fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [23]:
#print the coeefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.747454023064712,38.138628586523744,0.15535551583832705,60.93077134019082] Intercept: -1032.004219279001


In [28]:
test_results = lrModel.evaluate(test_data)

In [30]:
#Interesting results
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-19.165372243901288|
|-1.4163014355921177|
| 22.720150845263106|
|-1.4541973805434054|
| 1.6810675493191525|
+-------------------+
only showing top 5 rows



In [31]:
print("RMSE: {} ".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("r2: {}".format(test_results.r2))


RMSE: 10.459733167696118 
MSE: 109.40601793940229
r2: 0.986185075162016


In [32]:
#Check test dataset
test_model = lrModel.transform(test_data)

In [33]:
#Inspect results

In [34]:
test_model.select("Predict_Yearly Amount Spent", "Yearly Amount Spent").show(5)

+---------------------------+-------------------+
|Predict_Yearly Amount Spent|Yearly Amount Spent|
+---------------------------+-------------------+
|          285.2517131923703|   266.086340948469|
|          422.7429326925435|  421.3266312569514|
|          569.0609385804044|  591.7810894256675|
|          542.6807813698717|  541.2265839893283|
|         495.25237870621277|  496.9334462555319|
+---------------------------+-------------------+
only showing top 5 rows



In [35]:
# Save model
lrModel.save('lrModel_Ecommerce_Customers')

In [36]:
#Load model from
lrModel2 = LinearRegressionModel.load('lrModel_Ecommerce_Customers')

In [37]:
#Predict new values (Assuming select test_Data)
unlabeled_data = test_data.select('features')

In [38]:
predictions = lrModel2.transform(unlabeled_data)

In [39]:
predictions.show(5)

+--------------------+---------------------------+
|            features|Predict_Yearly Amount Spent|
+--------------------+---------------------------+
|[30.8162006488763...|          285.2517131923703|
|[31.2606468698795...|          422.7429326925435|
|[31.2834474760581...|          569.0609385804044|
|[31.5761319713222...|          542.6807813698717|
|[31.7366356860502...|         495.25237870621277|
+--------------------+---------------------------+
only showing top 5 rows



In [40]:
test_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.8162006488763...|   266.086340948469|
|[31.2606468698795...|  421.3266312569514|
|[31.2834474760581...|  591.7810894256675|
|[31.5761319713222...|  541.2265839893283|
|[31.7366356860502...|  496.9334462555319|
+--------------------+-------------------+
only showing top 5 rows

